In [1]:
import pandas as pd
import numpy as np
import os
import geopy.distance
import json
import requests

# from googlemaps import Client as GoogleMaps
from geopy.geocoders import Nominatim
from sklearn import preprocessing
from collections import namedtuple

In [3]:
data_path = "/Users/vincentchu/Dev/oakland-crime-prediction/data/"

data = os.path.join(os.getcwd(),data_path + 'crimePublicData.csv')
crime_data = pd.read_csv(data)#.to_sparse()

In [4]:
print "Shape of crime_data: ", crime_data.shape
crime_data.describe()

Shape of crime_data:  (8927, 8)


,CrimeType,DateTime,CaseNumber,Description,PoliceBeat,Address,City,State
count,8927,8923,8927,8927,8885,8927,8927,8927
unique,14,6704,8913,235,62,4603,1,1
top,THEFT/LARCENY,2018-01-01 00:00:00,17-067802,BURGLARY-AUTO,04X,400 7TH ST,Oakland,CA
freq,2695,12,2,1485,477,101,8927,8927


In [5]:
crime_data["CrimeType"].unique()

array(['MOTOR VEHICLE THEFT', 'ROBBERY', 'VANDALISM', 'THEFT/LARCENY',
       'ASSAULT', 'HOMICIDE', 'FRAUD', 'WEAPONS',
       'DRUGS/ALCOHOL VIOLATIONS', 'BURGLARY', 'SEX CRIMES', 'DUI',
       'DISTURBING THE PEACE', 'ARSON'], dtype=object)

In [6]:
crime_data.columns

Index([u'CrimeType', u'DateTime', u'CaseNumber', u'Description', u'PoliceBeat',
       u'Address', u'City', u'State'],
      dtype='object')

In [7]:
crime_data["CrimeType"].value_counts()

THEFT/LARCENY               2695
ASSAULT                     1820
MOTOR VEHICLE THEFT         1407
VANDALISM                    970
BURGLARY                     534
ROBBERY                      523
DRUGS/ALCOHOL VIOLATIONS     303
DISTURBING THE PEACE         242
FRAUD                        112
WEAPONS                       93
SEX CRIMES                    87
DUI                           77
ARSON                         47
HOMICIDE                      17
Name: CrimeType, dtype: int64

In [8]:
crime_data.head()

,CrimeType,DateTime,CaseNumber,Description,PoliceBeat,Address,City,State
0,MOTOR VEHICLE THEFT,2104-08-08 00:00:00,14-040258,VEHICLE THEFT - AUTO,02X,12TH ST & MANDELA PKWY,Oakland,CA
1,ROBBERY,2030-01-11 19:00:00,14-023460,"ROBBERY - STRONG ARM (HANDS, FISTS, FEET, ETC.)",23X,3600 BLK OF INTERNATIONAL BL,Oakland,CA
2,VANDALISM,2040-02-01 21:40:00,14-035766,"VANDALISM [UNDER $1,000] (AMENDED)",09X,600 FAIRMOUNT AV,Oakland,CA
3,THEFT/LARCENY,2050-01-03 13:55:00,14-038177,BURGLARY-AUTO,08X,TELEGRAPH AV & W MACARTHUR BLVD,Oakland,CA
4,ASSAULT,2018-03-15 03:50:00,18-013298,FORCE/ADW-OTHER DANGEROUS WEAPON:GBI,24Y,35TH AV & SUTER ST,Oakland,CA


In [9]:
# Adding columns for Lat, Long and Zip
crime_data['Lat'] = 0.0
crime_data['Long'] = 0.0
crime_data['Zip'] = 0

crime_data_copy = crime_data.copy()

In [10]:
# Default API key: replace with your own api key
google_api_key = ""

# Function to call Google's geocode api based on an address
# and returns a dictionary in json format
def get_geocode_json(address, api_key):
    geo_s ='https://maps.googleapis.com/maps/api/geocode/json'
    param = {'address': address, 'key': google_api_key }    
    response = requests.get(geo_s, params=param)
    result_dict = response.json()
    return result_dict    

In [22]:
# Get Lat/Long and Zip for all addresses in the current set of crime data

#for i in range(crime_data.shape[0]):
for i in range(0, 1000):    
    print "Iteration i = " + str(i)
    
    address = crime_data.at[crime_data.index[i], 'Address'] + ", " + crime_data.at[crime_data.index[i], 'City'] + ", " + crime_data.at[crime_data.index[i], 'State']
    result_dict = get_geocode_json(address, google_api_key)

    print "len(result_dict['results']) = %d"%(len(result_dict['results']))
    try:
        crime_data.at[crime_data.index[i], 'Lat'] = result_dict['results'][0]['geometry']['location']['lat']
    except:
        print "i = " + str(i) + ": Cannot find Latitude"
    
    try:
        crime_data.at[crime_data.index[i], 'Long'] = result_dict['results'][0]['geometry']['location']['lng']
    except:
        print "i = " + str(i) + ": Cannot find Longitude"
        
    try:    
        for j in range(len(result_dict['results'][0]['address_components'])):
            if result_dict['results'][0]['address_components'][j]['types'][0] == 'postal_code':
                crime_data.at[crime_data.index[i], 'Zip'] = result_dict['results'][0]['address_components'][j]['long_name']
    except:
        print "i = " + str(i) + ": Cannot find Address Components"

Iteration i = 0
len(result_dict['results']) = 1
Iteration i = 1
len(result_dict['results']) = 1
Iteration i = 2
len(result_dict['results']) = 1
Iteration i = 3
len(result_dict['results']) = 1
Iteration i = 4
len(result_dict['results']) = 1
Iteration i = 5
len(result_dict['results']) = 1
Iteration i = 6
len(result_dict['results']) = 1
Iteration i = 7
len(result_dict['results']) = 1
Iteration i = 8
len(result_dict['results']) = 1
Iteration i = 9
len(result_dict['results']) = 1
Iteration i = 10
len(result_dict['results']) = 1
Iteration i = 11
len(result_dict['results']) = 0
i = 11: Cannot find Latitude
i = 11: Cannot find Longitude
i = 11: Cannot find Address Components
Iteration i = 12
len(result_dict['results']) = 0
i = 12: Cannot find Latitude
i = 12: Cannot find Longitude
i = 12: Cannot find Address Components
Iteration i = 13
len(result_dict['results']) = 0
i = 13: Cannot find Latitude
i = 13: Cannot find Longitude
i = 13: Cannot find Address Components
Iteration i = 14
len(result_di

len(result_dict['results']) = 1
Iteration i = 141
len(result_dict['results']) = 1
Iteration i = 142
len(result_dict['results']) = 1
Iteration i = 143
len(result_dict['results']) = 1
Iteration i = 144
len(result_dict['results']) = 1
Iteration i = 145
len(result_dict['results']) = 1
Iteration i = 146
len(result_dict['results']) = 1
Iteration i = 147
len(result_dict['results']) = 1
Iteration i = 148
len(result_dict['results']) = 1
Iteration i = 149
len(result_dict['results']) = 1
Iteration i = 150
len(result_dict['results']) = 1
Iteration i = 151
len(result_dict['results']) = 1
Iteration i = 152
len(result_dict['results']) = 1
Iteration i = 153
len(result_dict['results']) = 1
Iteration i = 154
len(result_dict['results']) = 1
Iteration i = 155
len(result_dict['results']) = 0
i = 155: Cannot find Latitude
i = 155: Cannot find Longitude
i = 155: Cannot find Address Components
Iteration i = 156
len(result_dict['results']) = 1
Iteration i = 157
len(result_dict['results']) = 1
Iteration i = 158

len(result_dict['results']) = 1
Iteration i = 281
len(result_dict['results']) = 1
Iteration i = 282
len(result_dict['results']) = 1
Iteration i = 283
len(result_dict['results']) = 1
Iteration i = 284
len(result_dict['results']) = 1
Iteration i = 285
len(result_dict['results']) = 1
Iteration i = 286
len(result_dict['results']) = 2
Iteration i = 287
len(result_dict['results']) = 1
Iteration i = 288
len(result_dict['results']) = 1
Iteration i = 289
len(result_dict['results']) = 1
Iteration i = 290
len(result_dict['results']) = 1
Iteration i = 291
len(result_dict['results']) = 1
Iteration i = 292
len(result_dict['results']) = 1
Iteration i = 293
len(result_dict['results']) = 1
Iteration i = 294
len(result_dict['results']) = 1
Iteration i = 295
len(result_dict['results']) = 1
Iteration i = 296
len(result_dict['results']) = 1
Iteration i = 297
len(result_dict['results']) = 1
Iteration i = 298
len(result_dict['results']) = 1
Iteration i = 299
len(result_dict['results']) = 1
Iteration i = 300


len(result_dict['results']) = 1
Iteration i = 419
len(result_dict['results']) = 1
Iteration i = 420
len(result_dict['results']) = 1
Iteration i = 421
len(result_dict['results']) = 1
Iteration i = 422
len(result_dict['results']) = 1
Iteration i = 423
len(result_dict['results']) = 1
Iteration i = 424
len(result_dict['results']) = 1
Iteration i = 425
len(result_dict['results']) = 1
Iteration i = 426
len(result_dict['results']) = 1
Iteration i = 427
len(result_dict['results']) = 1
Iteration i = 428
len(result_dict['results']) = 1
Iteration i = 429
len(result_dict['results']) = 1
Iteration i = 430
len(result_dict['results']) = 1
Iteration i = 431
len(result_dict['results']) = 1
Iteration i = 432
len(result_dict['results']) = 1
Iteration i = 433
len(result_dict['results']) = 1
Iteration i = 434
len(result_dict['results']) = 1
Iteration i = 435
len(result_dict['results']) = 1
Iteration i = 436
len(result_dict['results']) = 0
i = 436: Cannot find Latitude
i = 436: Cannot find Longitude
i = 436

len(result_dict['results']) = 1
Iteration i = 559
len(result_dict['results']) = 1
Iteration i = 560
len(result_dict['results']) = 1
Iteration i = 561
len(result_dict['results']) = 1
Iteration i = 562
len(result_dict['results']) = 1
Iteration i = 563
len(result_dict['results']) = 1
Iteration i = 564
len(result_dict['results']) = 1
Iteration i = 565
len(result_dict['results']) = 0
i = 565: Cannot find Latitude
i = 565: Cannot find Longitude
i = 565: Cannot find Address Components
Iteration i = 566
len(result_dict['results']) = 1
Iteration i = 567
len(result_dict['results']) = 1
Iteration i = 568
len(result_dict['results']) = 0
i = 568: Cannot find Latitude
i = 568: Cannot find Longitude
i = 568: Cannot find Address Components
Iteration i = 569
len(result_dict['results']) = 1
Iteration i = 570
len(result_dict['results']) = 1
Iteration i = 571
len(result_dict['results']) = 0
i = 571: Cannot find Latitude
i = 571: Cannot find Longitude
i = 571: Cannot find Address Components
Iteration i = 5

len(result_dict['results']) = 1
Iteration i = 696
len(result_dict['results']) = 1
Iteration i = 697
len(result_dict['results']) = 1
Iteration i = 698
len(result_dict['results']) = 1
Iteration i = 699
len(result_dict['results']) = 1
Iteration i = 700
len(result_dict['results']) = 0
i = 700: Cannot find Latitude
i = 700: Cannot find Longitude
i = 700: Cannot find Address Components
Iteration i = 701
len(result_dict['results']) = 0
i = 701: Cannot find Latitude
i = 701: Cannot find Longitude
i = 701: Cannot find Address Components
Iteration i = 702
len(result_dict['results']) = 0
i = 702: Cannot find Latitude
i = 702: Cannot find Longitude
i = 702: Cannot find Address Components
Iteration i = 703
len(result_dict['results']) = 1
Iteration i = 704
len(result_dict['results']) = 1
Iteration i = 705
len(result_dict['results']) = 1
Iteration i = 706
len(result_dict['results']) = 1
Iteration i = 707
len(result_dict['results']) = 1
Iteration i = 708
len(result_dict['results']) = 0
i = 708: Cannot

len(result_dict['results']) = 1
Iteration i = 827
len(result_dict['results']) = 1
Iteration i = 828
len(result_dict['results']) = 1
Iteration i = 829
len(result_dict['results']) = 1
Iteration i = 830
len(result_dict['results']) = 1
Iteration i = 831
len(result_dict['results']) = 0
i = 831: Cannot find Latitude
i = 831: Cannot find Longitude
i = 831: Cannot find Address Components
Iteration i = 832
len(result_dict['results']) = 1
Iteration i = 833
len(result_dict['results']) = 1
Iteration i = 834
len(result_dict['results']) = 0
i = 834: Cannot find Latitude
i = 834: Cannot find Longitude
i = 834: Cannot find Address Components
Iteration i = 835
len(result_dict['results']) = 1
Iteration i = 836
len(result_dict['results']) = 1
Iteration i = 837
len(result_dict['results']) = 1
Iteration i = 838
len(result_dict['results']) = 1
Iteration i = 839
len(result_dict['results']) = 1
Iteration i = 840
len(result_dict['results']) = 1
Iteration i = 841
len(result_dict['results']) = 1
Iteration i = 84

len(result_dict['results']) = 1
Iteration i = 969
len(result_dict['results']) = 1
Iteration i = 970
len(result_dict['results']) = 1
Iteration i = 971
len(result_dict['results']) = 1
Iteration i = 972
len(result_dict['results']) = 1
Iteration i = 973
len(result_dict['results']) = 1
Iteration i = 974
len(result_dict['results']) = 1
Iteration i = 975
len(result_dict['results']) = 1
Iteration i = 976
len(result_dict['results']) = 1
Iteration i = 977
len(result_dict['results']) = 1
Iteration i = 978
len(result_dict['results']) = 1
Iteration i = 979
len(result_dict['results']) = 1
Iteration i = 980
len(result_dict['results']) = 1
Iteration i = 981
len(result_dict['results']) = 1
Iteration i = 982
len(result_dict['results']) = 1
Iteration i = 983
len(result_dict['results']) = 1
Iteration i = 984
len(result_dict['results']) = 1
Iteration i = 985
len(result_dict['results']) = 1
Iteration i = 986
len(result_dict['results']) = 1
Iteration i = 987
len(result_dict['results']) = 1
Iteration i = 988


In [37]:
# Create data frame with unique list of zip codes for Oakland
oakland_zip_codes = pd.DataFrame(transformed_crime_data.Zip.unique(), columns=['zip'])

In [42]:
# Get individual poverty data
data = os.path.join(os.getcwd(),data_path + 'ind_poverty_status_by_zip.csv')
ind_poverty_status = pd.read_csv(data)
ind_poverty_status = ind_poverty_status.drop(0, axis=0)
ind_poverty_status['zip'] = [int(ind_poverty_status.at[ind_poverty_status.index[i], 'GEO.id2']) for i in range(ind_poverty_status.shape[0])]
print ind_poverty_status.shape

# Filter out data not for Oakland
ind_poverty_status_oakland = ind_poverty_status.merge(oakland_zip_codes, 
                                                      left_on = 'zip', 
                                                      right_on = 'zip', 
                                                      how = 'inner')

print ind_poverty_status_oakland.shape
ind_poverty_status_oakland.head()

poverty_status = ind_poverty_status_oakland[['zip', 'HC03_EST_VC01', 'HC03_MOE_VC01']]

poverty_status

(1775, 370)
(35, 370)


,zip,HC03_EST_VC01,HC03_MOE_VC01
0,94025,7.8,1.3
1,94102,25.0,2.3
2,94123,5.1,1.0
3,94501,11.1,1.2
4,94505,6.0,2.0
5,94509,17.4,1.8
6,94526,3.4,1.1
7,94544,13.2,1.3
8,94551,6.3,1.4
9,94553,7.8,1.5


In [40]:
# Get family poverty data
data = os.path.join(os.getcwd(),data_path + 'family_poverty_status_by_zip.csv')
family_poverty_status = pd.read_csv(data)
family_poverty_status = family_poverty_status.drop(0, axis=0)
family_poverty_status['zip'] = [int(family_poverty_status.at[ind_poverty_status.index[i], 'GEO.id2']) for i in range(ind_poverty_status.shape[0])]
print family_poverty_status.shape

# Filter out data not for Oakland
family_poverty_status_oakland = family_poverty_status.merge(oakland_zip_codes, 
                                                            left_on = 'zip', 
                                                            right_on = 'zip', 
                                                            how = 'inner')

print family_poverty_status_oakland.shape
family_poverty_status_oakland.head()

(1775, 604)
(35, 604)


,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC03_EST_VC01,HC03_MOE_VC01,HC04_EST_VC01,...,HC02_MOE_VC73,HC03_EST_VC73,HC03_MOE_VC73,HC04_EST_VC73,HC04_MOE_VC73,HC05_EST_VC73,HC05_MOE_VC73,HC06_EST_VC73,HC06_MOE_VC73,zip
0,8600000US94025,94025,ZCTA5 94025,9734,296,4.8,1.3,7987,306,2.4,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),94025
1,8600000US94102,94102,ZCTA5 94102,4221,407,11.9,3.5,2597,346,9.5,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),94102
2,8600000US94123,94123,ZCTA5 94123,4526,372,2.8,1.4,3921,368,1.2,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),94123
3,8600000US94501,94501,ZCTA5 94501,15416,380,8.1,1.2,11288,444,5.5,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),94501
4,8600000US94505,94505,ZCTA5 94505,4075,201,4.3,2.0,3386,197,0.8,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),94505


In [76]:
# write crime_data to csv
crime_data.to_csv('crimePublicData_transformed.csv')

# Examine head of the data frame for validation
crime_data.head()

,CrimeType,DateTime,CaseNumber,Description,PoliceBeat,Address,City,State,Lat,Long,Zip
0,MOTOR VEHICLE THEFT,2104-08-08 00:00:00,14-040258,VEHICLE THEFT - AUTO,02X,12TH ST & MANDELA PKWY,Oakland,CA,37.809085,-122.291942,94607
1,ROBBERY,2030-01-11 19:00:00,14-023460,"ROBBERY - STRONG ARM (HANDS, FISTS, FEET, ETC.)",23X,3600 BLK OF INTERNATIONAL BL,Oakland,CA,37.775883,-122.221834,94601
2,VANDALISM,2040-02-01 21:40:00,14-035766,"VANDALISM [UNDER $1,000] (AMENDED)",09X,600 FAIRMOUNT AV,Oakland,CA,37.822005,-122.253938,94611
3,THEFT/LARCENY,2050-01-03 13:55:00,14-038177,BURGLARY-AUTO,08X,TELEGRAPH AV & W MACARTHUR BLVD,Oakland,CA,37.826355,-122.265215,94609
4,FRAUD,2018-02-07 22:15:00,18-902981,USE ANOTHER'S PERSONAL IDENTIFICATION TO OBTAI...,13Z,00 VISTA DEL MAR CT,Oakland,CA,37.837958,-122.205544,94611


***Testing Section***

In [77]:
# # Print out Lat/Long and Zip of part of the data frame for validation
# for i in range(0, 5):
#     print "Iteration %d" % i
#     print "(Lat, Long) = ", (crime_data.at[crime_data.index[i], 'Lat'], crime_data.at[crime_data.index[i], 'Long'])
#     print "Zip = %d" % crime_data.at[crime_data.index[i], 'Zip']

Iteration 0
(Lat, Long) =  (37.80908470000001, -122.2919421)
Zip = 94607
Iteration 1
(Lat, Long) =  (37.7758827, -122.2218341)
Zip = 94601
Iteration 2
(Lat, Long) =  (37.8220053, -122.2539378)
Zip = 94611
Iteration 3
(Lat, Long) =  (37.8263547, -122.2652151)
Zip = 94609
Iteration 4
(Lat, Long) =  (37.8379578, -122.2055437)
Zip = 94611


In [26]:
# data_path = "/Users/vincentchu/Dev/oakland-crime-prediction/data/"

# data = os.path.join(os.getcwd(),data_path + 'crimePublicData_transformed.csv')
# transformed_crime_data = pd.read_csv(data)#.to_sparse()

In [28]:
transformed_crime_data.Zip.unique()

array([94607, 94601, 94611, 94609, 94603, 94621, 94605, 94606, 94610,
       94618, 94602, 94608, 94612, 94619,     0, 94705, 94577, 94660,
       94553, 94505, 94708, 94702, 94102, 94501, 94806, 94563, 94526,
       94703, 94615, 94704, 94123, 94720, 94613, 94025, 94544, 94509,
       94551, 94801])